Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference TensorFlow Bert Model with ONNX Runtime on CPU

In this tutorial, you'll be introduced to how to load a Bert model using TensorFlow, convert it to ONNX using Keras2onnx, and inference it for high performance using ONNX Runtime. In the following sections, we are going to use the Bert model trained with Stanford Question Answering Dataset (SQuAD) dataset as an example. Bert SQuAD model is used in question answering scenarios, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

## 0. Prerequisites ##
First we need a python environment before running this notebook.

You can install [AnaConda](https://www.anaconda.com/distribution/) and [Git](https://git-scm.com/downloads) and open an AnaConda console when it is done. Then you can run the following commands to create a conda environment named cpu_env:

```console
conda create -n cpu_env python=3.6
conda activate cpu_env

conda install -c anaconda ipykernel
conda install -c conda-forge ipywidgets
python -m ipykernel install --user --name=cpu_env
```

Finally, launch Jupyter Notebook and you can choose cpu_env as kernel to run this notebook.

Let's install [Tensorflow](https://www.tensorflow.org/install), [OnnxRuntime](https://microsoft.github.io/onnxruntime/), Keras2Onnx and other packages like the following:

In [1]:
import sys

!{sys.executable} -m pip install --quiet --upgrade tensorflow==2.1.0
!{sys.executable} -m pip install --quiet --upgrade onnxruntime

# Install keras2onnx from source, since the latest package (1.6.0) does not support bert models from tensorflow 2.1 currently.
!{sys.executable} -m pip install --quiet git+https://github.com/microsoft/onnxconverter-common
!{sys.executable} -m pip install --quiet git+https://github.com/onnx/keras-onnx
    
# Install other packages used in this notebook.   
!{sys.executable} -m pip install --quiet transformers==2.5.1
!{sys.executable} -m pip install --quiet wget psutil onnx pytz pandas py-cpuinfo py3nvml

In [2]:
# Whether allow overwrite existing script or model.
enable_overwrite = True

# Number of runs to get average latency.
total_runs = 100

In [3]:
import os
import wget

cache_dir = "./squad"
output_dir = "./output"
script_dir = './bert_scripts'

for directory in [cache_dir, output_dir, script_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Download scripts for BERT optimization.
url_prfix = "https://raw.githubusercontent.com/microsoft/onnxruntime/master/onnxruntime/python/tools/bert/"
script_files = ['bert_perf_test.py', 'bert_test_data.py', 'compare_bert_results.py', 'BertOnnxModel.py', 'BertOnnxModelKeras.py', 'BertOnnxModelTF.py', 'Gpt2OnnxModel.py', 'OnnxModel.py', 'bert_model_optimization.py']

for filename in script_files:
    target_file = os.path.join(script_dir, filename)
    if enable_overwrite and os.path.exists(target_file):
        os.remove(target_file)
    if not os.path.exists(target_file):
        wget.download(url_prfix + filename, target_file)
        print("Downloaded", filename)

100% [..............................................................................] 15310 / 15310Downloaded bert_perf_test.py
100% [................................................................................] 9571 / 9571Downloaded bert_test_data.py
100% [................................................................................] 7272 / 7272Downloaded compare_bert_results.py
100% [..............................................................................] 44905 / 44905Downloaded BertOnnxModel.py
100% [..............................................................................] 21565 / 21565Downloaded BertOnnxModelKeras.py
100% [..............................................................................] 26114 / 26114Downloaded BertOnnxModelTF.py
100% [..............................................................................] 22773 / 22773Downloaded OnnxModel.py
100% [................................................................................] 7795 / 7795

## 1. Load Pretrained Bert model ##

Start to load fine-tuned model. This step take a few minutes to download the model (1.3 GB) for the first time.

In [4]:
from transformers import (TFBertForQuestionAnswering, BertTokenizer)

model_name_or_path = 'bert-large-uncased-whole-word-masking-finetuned-squad'

# Load model and tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name_or_path, do_lower_case=True, cache_dir=cache_dir)
model = TFBertForQuestionAnswering.from_pretrained(model_name_or_path, cache_dir=cache_dir)

## 2. TensorFlow Inference

Use one example to run inference using TensorFlow as baseline.

In [5]:
import numpy
question, text = "What is ONNX Runtime?", "ONNX Runtime is a performance-focused inference engine for ONNX models."
inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors='tf')

start_scores, end_scores = model(inputs)

num_tokens = len(inputs["input_ids"][0])
all_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
print("The answer is:", ' '.join(all_tokens[numpy.argmax(start_scores) : numpy.argmax(end_scores)+1]))

The answer is: a performance - focused inference engine for on ##nx models


In [6]:
import time
start = time.time()
for _ in range(total_runs):
    start_scores, end_scores = model(inputs)
end = time.time()
print("Tensorflow Inference time for sequence length {} = {} ms".format(num_tokens, format((end - start) * 1000 / total_runs, '.2f')))

Tensorflow Inference time for sequence length 26 = 227.06 ms


## 3. Export model to ONNX using Keras2onnx

Now we use Keras2onnx to export the model to ONNX format. It takes about 18 minutes for the large model.

In [7]:
import keras2onnx

output_model_path =  os.path.join(output_dir, 'keras_{}.onnx'.format(model_name_or_path))

if enable_overwrite or not os.path.exists(output_model_path):
    model.predict(inputs)
    start = time.time()
    onnx_model = keras2onnx.convert_keras(model, model.name)
    keras2onnx.save_model(onnx_model, output_model_path)
    print("Keras2onnx run time = {} s".format(format(time.time() - start, '.2f')))

The node number after optimization: 5257 -> 3836


Keras2onnx run time = 1052.26 s


## 4. Inference the Exported Model with ONNX Runtime

### OpenMP Environment Variable

OpenMP environment variable is important for CPU inference of Bert models. After running this notebook, you can find the best setting from [Performance Test Tool](#Performance-Test-Tool) result for your machine.

Setting environment variables shall be done before importing onnxruntime. Otherwise, they might not take effect.

In [8]:
import os
import psutil

# You may change the settings in this cell according to Performance Test Tool result after running the whole notebook.
use_openmp = True

# ATTENTION: these environment variables must be set before importing onnxruntime.
if use_openmp:
    os.environ["OMP_NUM_THREADS"] = str(psutil.cpu_count(logical=True))
else:
    os.environ["OMP_NUM_THREADS"] = '1'

os.environ["OMP_WAIT_POLICY"] = 'ACTIVE'

Now we are ready to inference the model with ONNX Runtime. Here we can see that OnnxRuntime has better performance than TensorFlow for this example even without optimization.

In [9]:
import psutil
import onnxruntime
import numpy

# User might use onnxruntime-gpu for CPU inference.
if use_openmp and 'CUDAExecutionProvider' in onnxruntime.get_available_providers():
    print("warning: onnxruntime-gpu is not built with OpenMP. You might try onnxruntime package.")
   
sess_options = onnxruntime.SessionOptions()

# The following settings enables OpenMP, which is required to get best performance for CPU inference of Bert models.
if use_openmp:
    sess_options.intra_op_num_threads=1
else:
    sess_options.intra_op_num_threads=psutil.cpu_count(logical=True)

# Providers is optional. Only needed when you use onnxruntime-gpu for CPU inference.
session = onnxruntime.InferenceSession(output_model_path, sess_options, providers=['CPUExecutionProvider'])

# Use contiguous array as input could improve performance.
inputs_onnx = {k_: numpy.ascontiguousarray(v_.numpy()) for k_, v_ in inputs.items()}

# Warm up with one run.
results = session.run(None, inputs_onnx)

# Measure the latency.
start = time.time()
for _ in range(total_runs):
    results = session.run(None, inputs_onnx)
end = time.time()
print("ONNX Runtime cpu inference time for sequence length {} (model not optimized): {} ms".format(num_tokens, format((end - start) * 1000 / total_runs, '.2f')))
del session

ONNX Runtime cpu inference time for sequence length 26 (model not optimized): 170.70 ms


In [10]:
print("***** Verifying correctness (TensorFlow and ONNX Runtime) *****")
print('start_scores are close:', numpy.allclose(results[0], start_scores.cpu(), rtol=1e-05, atol=1e-04))
print('end_scores are close:', numpy.allclose(results[1], end_scores.cpu(), rtol=1e-05, atol=1e-04))

***** Verifying correctness (TensorFlow and ONNX Runtime) *****
Instructions for updating:
Use tf.identity instead.
start_scores are close: True
end_scores are close: True


## 5. Model Optimization

[ONNX Runtime BERT Model Optimization Tools](https://github.com/microsoft/onnxruntime/tree/master/onnxruntime/python/tools/bert) is a set of tools for optimizing and testing BERT models. Let's try some of them on the exported models.

### BERT Optimization Script

The script **bert_model_optimization.py** can help optimize BERT model exported by PyTorch, tf2onnx or keras2onnx. Since our model is exported by keras2onnx, we shall use **--model_type bert_keras** parameter.

It will also tell whether the model is fully optimized or not. If not, that means you might need change the script to fuse some new pattern of subgraph.

In [11]:
optimized_model_path = os.path.join(output_dir, 'keras_bert_large_opt_cpu.onnx')

%run bert_scripts/bert_model_optimization.py --input $output_model_path --output $optimized_model_path --model_type bert_keras --num_heads 16 --hidden_size 1024

  BertOnnxModelTF.py: Fused LayerNormalization count: 49
BertOnnxModelKeras.py: Fused Gelu count:24
BertOnnxModelKeras.py: start processing embedding layer...
BertOnnxModelKeras.py: Found word embedding. name:tf_bert_for_question_answering/bert/embeddings/Gather/resource:0, shape:(30522, 1024)
BertOnnxModelKeras.py: Found word embedding. name:tf_bert_for_question_answering/bert/embeddings/position_embeddings/embedding_lookup/413066:0, shape:(512, 1024)
BertOnnxModelKeras.py: Found segment embedding. name:tf_bert_for_question_answering/bert/embeddings/token_type_embeddings/embedding_lookup/413071:0, shape:(2, 1024)
BertOnnxModelKeras.py: Create Embedding node
        OnnxModel.py: Graph pruned: 0 inputs, 0 outputs and 9 nodes are removed
BertOnnxModelKeras.py: Fused mask
BertOnnxModelKeras.py: Skip consequent Reshape count: 24
    BertOnnxModel.py: Fused Reshape count:0
    BertOnnxModel.py: Fused SkipLayerNormalization count: 48
BertOnnxModelKeras.py: Fused Attention count:24
    BertO

We run the optimized model using same inputs. The inference latency is reduced after optimization. The output result is the same as the one before optimization.

In [12]:
session = onnxruntime.InferenceSession(optimized_model_path, sess_options)
# use one run to warm up a session
session.run(None, inputs_onnx)

# measure the latency.
start = time.time()
for _ in range(total_runs):
    opt_results = session.run(None, inputs_onnx)
end = time.time()
print("ONNX Runtime cpu inference time on optimized model: {} ms".format(format((end - start) * 1000 / total_runs, '.2f')))
del session

ONNX Runtime cpu inference time on optimized model: 133.35 ms


In [13]:
print("***** Verifying correctness (before and after optimization) *****")
print('start_scores are close:', numpy.allclose(opt_results[0], start_scores.cpu(), rtol=1e-05, atol=1e-04))
print('end_scores are close:', numpy.allclose(opt_results[1], end_scores.cpu(), rtol=1e-05, atol=1e-04))

***** Verifying correctness (before and after optimization) *****
start_scores are close: True
end_scores are close: True


### Model Results Comparison Tool

If your BERT model has three inputs, a script compare_bert_results.py can be used to do a quick verification. The tool will generate some fake input data, and compare results from both the original and optimized models. If outputs are all close, it is safe to use the optimized model.

Example of comparing the models before and after optimization:

In [14]:
# The base model is exported using sequence length 26
%run ./bert_scripts/compare_bert_results.py --baseline_model $output_model_path --optimized_model $optimized_model_path --batch_size 1 --sequence_length 26 --samples 10

100% passed for 10 random inputs given thresholds (rtol=0.001, atol=0.0001).
maximum absolute difference=2.3484230041503906e-05
maximum relative difference=0.00013404049968812615


### Performance Test Tool

This tool measures performance of BERT model inference using OnnxRuntime Python API.

The following command will create 100 samples of batch_size 1 and sequence length 128 to run inference, then calculate performance numbers like average latency and throughput etc. 

It takes about 20 minutes to run this test. You can remove --all to reduce number of settings in the test.

In [15]:
%run ./bert_scripts/bert_perf_test.py --model $optimized_model_path --batch_size 1 --sequence_length 128 --samples 100 --test_times 1 --inclusive --all

Generating 100 samples for batch_size=1 sequence_length=128
Extra latency for converting inputs to contiguous: 0.04 ms
Test summary is saved to output\perf_results_CPU_B1_S128_20200319-141051.txt


Let's load the summary file and take a look. In this machine, the best result is achieved by OpenMP. The best setting might be difference using different hardware or model.

In [16]:
import glob     
import pandas

latest_result_file = max(glob.glob(os.path.join(output_dir, "perf_results_*.txt")), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file, converters={'OMP_NUM_THREADS': str, 'OMP_WAIT_POLICY':str})
print(latest_result_file)
print("The best setting is: {} openmp; {} contiguous array".format('use' if result_data['intra_op_num_threads'].iloc[0] == 1 else 'NO', 'use' if result_data['contiguous'].iloc[0] else 'NO'))

result_data.drop(['model', 'graph_optimization_level', 'batch_size', 'sequence_length', 'test_cases', 'test_times', 'use_gpu', 'warmup'], axis=1, inplace=True)
result_data.drop(['Latency_P50', 'Latency_P75', 'Latency_P90', 'Latency_P95'], axis=1, inplace=True)
cols = result_data.columns.tolist()
cols = cols[-4:] + cols[:-4]
result_data = result_data[cols]
result_data

./output\perf_results_CPU_B1_S128_20200319-141051.txt
The best setting is: use openmp; NO contiguous array


,intra_op_num_threads,OMP_NUM_THREADS,OMP_WAIT_POLICY,contiguous,Latency(ms),Latency_P99,Throughput(QPS)
0,1,12,PASSIVE,False,254.20,277.38,3.93
1,1,12,PASSIVE,True,255.47,283.56,3.91
2,1,12,ACTIVE,False,274.95,334.49,3.64
3,12,1,ACTIVE,True,278.91,294.30,3.59
4,1,12,ACTIVE,True,280.97,351.41,3.56
5,12,1,ACTIVE,False,281.89,296.11,3.55
6,12,1,PASSIVE,True,282.75,313.10,3.54
7,1,,,False,284.61,356.98,3.51
8,1,,,True,292.11,361.00,3.42
9,1,6,PASSIVE,False,292.28,346.47,3.42


## 6. Additional Info

Note that running Jupyter Notebook has slight impact on performance result since Jupyter Notebook is using system resources like CPU and memory etc. It is recommended to close Jupyter Notebook and other applications, then run the performance test tool in a console to get more accurate performance numbers.

[OnnxRuntime C API](https://github.com/microsoft/onnxruntime/blob/master/docs/C_API.md) could get slightly better performance than python API. If you use C API in inference, you can use OnnxRuntime_Perf_Test.exe built from source to measure performance instead.

Here is the machine configuration that generated the above results. The machine has GPU but not used in CPU inference.
You might get slower or faster result based on your hardware.

In [17]:
%run ./bert_scripts/MachineInfo.py --silent

{
  "gpu": {
    "driver_version": "441.22",
    "devices": [
      {
        "memory_total": 8589934592,
        "memory_available": 611880960,
        "name": "GeForce GTX 1070"
      }
    ]
  },
  "cpu": {
    "brand": "Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz",
    "cores": 6,
    "logical_cores": 12,
    "hz": "3.1920 GHz",
    "l2_cache": "1536 KB",
    "l3_cache": "12288 KB",
    "processor": "Intel64 Family 6 Model 158 Stepping 10, GenuineIntel"
  },
  "memory": {
    "total": 16971259904,
    "available": 6245142528
  },
  "python": "3.6.10.final.0 (64 bit)",
  "os": "Windows-10-10.0.18362-SP0",
  "onnxruntime": {
    "version": "1.2.0",
    "support_gpu": false
  },
  "pytorch": {
    "version": "1.4.0+cpu",
    "support_gpu": false
  },
  "tensorflow": {
    "version": "2.1.0",
    "git_version": "v2.1.0-rc2-17-ge5bf8de410",
    "support_gpu": true
  }
}
